In [1]:
import numpy as np
import os
import scipy.io as sio
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance
from matplotlib import pyplot as plt

In [4]:
# First, run the code in extract_precision_matrices.ipynb to obtain precision matrices: FCprec_concat_allsub.mat
# Then, run unpack_precision_mats to split FCprec_concat_allsub.mat into single subject .mat files

cwd='/Users/emilyolafson/GIT/stroke-graph-matching/data/'
unpack_precision_mats(cwd)

# After unpack_precision_mats is run, format precision data:
# - Set diagonals of precision FC to 0.
# - create a .mat file that contains precision FCs in a cell array. Subjects in rows & longitudinal time points in columns.

# MATLAB code for formatting this dataset: format_precision.m

# Then, 

In [9]:
def unpack_precision_mats(cwd):
    """Turn concatenated precision FC matrix into multiple single subject/single session matrices.
    Save each single suject to /precision/ folder."""
    data_dir = str(cwd) 
    subj_all = np.genfromtxt(data_dir + 'precision/subjects.txt', dtype = 'str') #list of subjects in same order as precision matrices are saved.    
    fc_dir = data_dir + 'precision/'
    output_prec = sio.loadmat(fc_dir + 'FCprec_concat_allsub.mat')
    prec = output_prec['C'] 
    c = 0
    for sub in subj_all:
        mdic = {"C": prec[c]}
        sio.savemat(fc_dir + sub, mdic)
        c=c+1
        
def remap_cost_regularized(interval, fc, alpha, beta):
    """Cost function: the cost of remapping each node in matrix A to every other node in matrix B. 
    Output is fed into graph matching algorithm. Regularization: (costmatrix - alpha*I) 
    where alpha is a regularization parameter controlling the degree to which a remapping
    to itself is preferred over remapping to any other node."""
    cwd = os.getcwd()
    data_dir = '/Users/emilyolafson/GIT/stroke-graph-matching/data'
    distances=sio.loadmat(data_dir + '/pairwise_eucl_dist_shen268.mat')

    distances=distances['eucl_dist']
    costmat_all=list()

    nROIs=fc[0][0].shape[0]
    
    #calculate cost between all ROIs
    for i in range(0,fc.shape[0]): #subject 

        if (dx == 'stroke' and i==5 and interval == [3,4]): #sub6 has no 4,5
            continue 
        if (dx == 'stroke' and i==11 and (interval == [3,4] or interval == [2,3])): #sub12 has no 4,5 OR 3,4
            continue
        if (dx == 'stroke' and i==19 and (interval == [3,4] or interval == [2,3] or interval == [1,2])): #sub20 has no 4,5 OR 3,4 OR 2,3
            continue
            
        costmat=np.zeros((nROIs,nROIs))

        for x in range(0,nROIs): #x = time point 1.
            a=fc[i,interval[0]][x]

            for y in range(0,nROIs): #y = time point 2.
                b=fc[i,interval[1]][y]
                
                costmat[x,y]=distance.euclidean(a,b)
                
        costmat=costmat+beta*distances
        costmat_all.append(costmat)

    return [costmat_all, nROIs]

def graph_matching_regularized(costmat_all, interval, nROIs, dx, alpha, alphacounter, beta, betacounter):
    """Runs graph matching w/ the Hungarian algorithm and saves outputs:
    
        cols_SXSY.txt - each row is a different subject. Values in each column represent the node
        in the latter time point that the node in the prior time point was mapped to.
        
        roichanges_SXSY.txt - each row is a different subject. Values in each column represent 
        whether the node was remapped to a DIFFERENT node than itself in the subsequent time point 
        (elements are 0 or 1, 1 if the node was remapped to a different node and 0 if the node was mapped to itself.
        Used to calculate remap frequency for each region, when averaged vertically (across subjects)."""
    
    results_dir = '/Users/emilyolafson/GIT/stroke-graph-matching/project/results/stroke'
    print(alphacounter)
    nROIs=fc[0][0].shape[0]
    nsubs=len(costmat_all) #test
    
    rows=np.zeros((nsubs,nROIs))
    cols=np.zeros((nsubs,nROIs))
    roichanges=np.zeros((nsubs,nROIs))
    truecols=range(0,nROIs)

    for i in range(0, nsubs):
        cost=costmat_all[i]
        print(cost*10000)
        rowind, colind=linear_sum_assignment(cost*10000) #graph matching part.
        rows[i]=rowind
        cols[i]=colind

    np.savetxt(results_dir+'/precision/cols_'+ 'S'+str(interval[0]+1)+'S'+ str(interval[1]+1)+'_alpha' + str(alphacounter)+'_beta' + str(betacounter) + '.txt', cols)

    for j in range(0,nsubs):
        for i in range(0,nROIs):
            if cols[j][i]!=truecols[i]:
                roichanges[j][i]=1 #indices that are switched

    allchanges=np.mean(roichanges,0)
    np.savetxt(results_dir +'/precision/roichanges_' + 'S'+str(interval[0]+1)+'S'+ str(interval[1]+1) +'_alpha' + str(alphacounter)+'_beta' + str(betacounter) +'.txt', allchanges)
    return [allchanges]

def run_graph_matching_reg(interval, fc, dx, alpha,alphacounter, beta, betacounter):
    """Calculate the cost of remapping across all subjects, and perform graph matching.
    Saves outputs to /results/jupyter/.../stroke and /controls."""
    [costmat_all, nROIs]=remap_cost_regularized(interval, fc, alpha, beta)
    nswaps=graph_matching_regularized(costmat_all, interval, nROIs, dx, alpha,alphacounter, beta, betacounter)
    return nswaps


In [10]:
cwd = os.getcwd()
data_dir = '/Users/emilyolafson/GIT/stroke-graph-matching/data'
fc_dir =data_dir + '/precision'
fc=sio.loadmat(fc_dir + '/C_precision.mat')
dx='stroke'
fc=fc['C_precision']
intervals=[[0, 1], [1, 2], [2, 3], [3, 4]] #time point comparisons. 0=1, 1=2, etc.
alpha=0
session_swaps=list()
for i in range(0, 4):
    allswaps=list()
    betas=[0, 0.0001, 0.0002, 0.0003, 0.0004]
    betacounter = -1
    alphacounter = 0
    for beta in betas:
        betacounter = betacounter +1
        print('Testing beta: ' + str(beta))
        nswaps=run_graph_matching_reg(intervals[i], fc, dx, alpha, alphacounter, beta, betacounter)
        print('Mean num. swaps: ' + str(np.mean(nswaps)))
        allswaps.append(nswaps)

    
    session_swaps.append(allswaps)

Testing beta: 0
0
[[196.85809623 411.72259928 272.23176571 ... 294.05084617 287.22149959
  289.02611093]
 [261.61986195 297.42615438 269.42144559 ... 288.24268458 293.40035852
  294.7231522 ]
 [248.18284233 397.73343518 175.45403697 ... 293.09149037 301.2149891
  302.52483228]
 ...
 [243.55115175 396.47373945 241.36813849 ... 104.33654361  67.55382577
   68.05236939]
 [231.79834925 406.3791365  239.53726631 ... 109.70387607  12.10536785
   10.11494304]
 [231.74770168 409.55596485 239.78142951 ... 110.04381777  14.98133308
   11.55853623]]
[[642.91422123 460.43003544 578.14591453 ... 566.61535065 559.03848195
  551.16876655]
 [756.5814563  297.10484714 540.59918189 ... 390.19796611 337.79118798
  333.2983928 ]
 [731.51896098 503.37581526 402.89678356 ... 612.26250591 610.39122867
  604.27515425]
 ...
 [847.09118559 411.85457845 602.37097744 ... 288.67804631 267.11300981
  262.79380211]
 [861.86041118 394.60991236 590.96449677 ... 224.74615138  54.00752039
   70.76316149]
 [879.00921517 

0
[[196.85809623 416.9535549  281.22452705 ... 313.0473522  309.6414883
  310.26306416]
 [266.85081756 297.42615438 273.18325132 ... 302.00823501 310.58939161
  310.72914982]
 [257.17560366 401.4952409  175.45403697 ... 303.09523507 314.64221647
  314.76902417]
 ...
 [262.54765778 410.23928987 251.37188319 ... 104.33654361  70.97730844
   70.29281659]
 [254.21833795 423.56816959 252.96449368 ... 113.12735874  12.10536785
   11.29797852]
 [252.98465491 425.56196247 252.02562141 ... 112.28426497  16.16436855
   11.55853623]]
[[642.91422123 465.66099105 587.13867587 ... 585.61185668 581.45847066
  572.40571978]
 [761.81241191 297.10484714 544.36098761 ... 403.96351653 354.98022108
  349.30439042]
 [740.51172232 507.13762099 402.89678356 ... 622.26625061 623.81845604
  616.51934614]
 ...
 [866.08769162 425.62012887 612.37472213 ... 288.67804631 270.53649248
  265.03424931]
 [884.28039989 411.79894546 604.39172414 ... 228.16963406  54.00752039
   71.94619697]
 [900.2461684  417.3726957  609

0
[[196.85809623 422.18451051 290.21728839 ... 332.04385823 332.061477
  331.50001739]
 [272.08177318 297.42615438 276.94505704 ... 315.77378543 327.77842471
  326.73514744]
 [266.168365   405.25704663 175.45403697 ... 313.09897976 328.06944384
  327.01321606]
 ...
 [281.54416381 424.00484029 261.37562788 ... 104.33654361  74.40079112
   72.53326379]
 [276.63832666 440.75720269 266.39172105 ... 116.55084142  12.10536785
   12.48101399]
 [274.22160814 441.56796009 264.2698133  ... 114.52471217  17.34740403
   11.55853623]]
[[642.91422123 470.89194666 596.1314372  ... 604.60836271 603.87845936
  593.64267301]
 [767.04336752 297.10484714 548.12279334 ... 417.72906695 372.16925418
  365.31038804]
 [749.50448366 510.89942671 402.89678356 ... 632.2699953  637.24568341
  628.76353804]
 ...
 [885.08419765 439.38567929 622.37846683 ... 288.67804631 273.95997516
  267.2746965 ]
 [906.7003886  428.98797855 617.81895151 ... 231.59311673  54.00752039
   73.12923244]
 [921.48312164 433.37869332 621.

0
[[196.85809623 427.41546612 299.21004972 ... 351.04036426 354.48146571
  352.73697062]
 [277.31272879 297.42615438 280.70686277 ... 329.53933585 344.9674578
  342.74114506]
 [275.16112634 409.01885235 175.45403697 ... 323.10272446 341.49667121
  339.25740796]
 ...
 [300.54066984 437.77039071 271.37937258 ... 104.33654361  77.82427379
   74.77371099]
 [299.05831537 457.94623579 279.81894842 ... 119.97432409  12.10536785
   13.66404947]
 [295.45856137 457.57395771 276.5140052  ... 116.76515937  18.53043951
   11.55853623]]
[[642.91422123 476.12290227 605.12419854 ... 623.60486874 626.29844807
  614.87962624]
 [772.27432313 297.10484714 551.88459906 ... 431.49461738 389.35828727
  381.31638566]
 [758.49724499 514.66123244 402.89678356 ... 642.27374    650.67291078
  641.00772993]
 ...
 [904.08070368 453.15122971 632.38221152 ... 288.67804631 277.38345783
  269.5151437 ]
 [929.1203773  446.17701165 631.24617888 ... 235.01659941  54.00752039
   74.31226792]
 [942.72007487 449.38469094 633

0
[[196.85809623 432.64642173 308.20281106 ... 370.0368703  376.90145442
  373.97392385]
 [282.5436844  297.42615438 284.46866849 ... 343.30488627 362.1564909
  358.74714268]
 [284.15388767 412.78065808 175.45403697 ... 333.10646915 354.92389858
  351.50159985]
 ...
 [319.53717588 451.53594113 281.38311727 ... 104.33654361  81.24775647
   77.01415819]
 [321.47830408 475.13526888 293.24617579 ... 123.39780677  12.10536785
   14.84708494]
 [316.6955146  473.57995532 288.75819709 ... 119.00560656  19.71347498
   11.55853623]]
[[642.91422123 481.35385788 614.11695988 ... 642.60137478 648.71843678
  636.11657947]
 [777.50527874 297.10484714 555.64640479 ... 445.2601678  406.54732037
  397.32238328]
 [767.49000633 518.42303816 402.89678356 ... 652.27748469 664.10013815
  653.25192183]
 ...
 [923.07720972 466.91678013 642.38595622 ... 288.67804631 280.80694051
  271.7555909 ]
 [951.54036601 463.36604474 644.67340625 ... 238.44008208  54.00752039
   75.49530339]
 [963.9570281  465.39068856 646

...

0
[[184.28253873 231.1579351  196.83508836 ... 224.08625197 232.63124055
  235.22596829]
 [351.3500147  265.769757   357.27329959 ... 385.33577235 394.38208137
  394.66953588]
 [240.21837928 225.18963352 158.11766291 ... 245.84799121 257.79490883
  259.9957917 ]
 ...
 [218.85264702 195.38060474 171.26609072 ...  95.26412927  60.35865288
   59.59932369]
 [219.28537613 209.08358595 161.06413569 ...  75.50335524   9.98061271
    8.99625973]
 [220.1275344  210.55568898 160.41980405 ...  75.11741719   8.59292751
    8.2779944 ]]
[[618.91544283 557.48495927 606.83861853 ... 726.46321628 633.61595531
  643.59634769]
 [774.28508377 377.18450023 570.5942754  ... 480.94966012 293.09531096
  313.07653305]
 [718.06247242 532.36144068 382.19796979 ... 687.71307133 593.38379145
  600.01773536]
 ...
 [834.12932821 415.66009602 667.62224679 ... 360.46881652 252.67470539
  272.11447036]
 [855.41916486 386.42739271 655.12057505 ... 361.33387389  78.35759038
  124.8993175 ]
 [860.65546197 391.34326723 65

0
[[184.28253873 236.38889071 205.8278497  ... 243.082758   255.05122926
  256.46292152]
 [356.58097031 265.769757   361.03510532 ... 399.10132277 411.57111446
  410.6755335 ]
 [249.21114061 228.95143925 158.11766291 ... 255.85173591 271.2221362
  272.2399836 ]
 ...
 [237.84915305 209.14615516 181.26983542 ...  95.26412927  63.78213556
   61.83977089]
 [241.70536483 226.27261904 174.49136306 ...  78.92683792   9.98061271
   10.17929521]
 [241.36448763 226.5616866  172.66399594 ...  77.35786439   9.77596298
    8.2779944 ]]
[[618.91544283 562.71591488 615.83137986 ... 745.45972232 656.03594402
  664.83330092]
 [779.51603938 377.18450023 574.35608113 ... 494.71521054 310.28434405
  329.08253067]
 [727.05523376 536.1232464  382.19796979 ... 697.71681602 606.81101882
  612.26192725]
 ...
 [853.12583425 429.42564644 677.62599149 ... 360.46881652 256.09818807
  274.35491756]
 [877.83915357 403.61642581 668.54780242 ... 364.75735656  78.35759038
  126.08235298]
 [881.8924152  407.34926485 667

0
[[184.28253873 241.61984632 214.82061104 ... 262.07926404 277.47121797
  277.69987475]
 [361.81192592 265.769757   364.79691104 ... 412.86687319 428.76014756
  426.68153111]
 [258.20390195 232.71324498 158.11766291 ... 265.8554806  284.64936357
  284.48417549]
 ...
 [256.84565908 222.91170558 191.27358011 ...  95.26412927  67.20561823
   64.08021809]
 [264.12535354 243.46165214 187.91859043 ...  82.35032059   9.98061271
   11.36233068]
 [262.60144086 242.56768422 184.90818783 ...  79.59831159  10.95899846
    8.2779944 ]]
[[618.91544283 567.94687049 624.8241412  ... 764.45622835 678.45593273
  686.07025416]
 [784.74699499 377.18450023 578.11788686 ... 508.48076096 327.47337715
  345.08852829]
 [736.0479951  539.88505213 382.19796979 ... 707.72056072 620.23824619
  624.50611914]
 ...
 [872.12234028 443.19119686 687.62973618 ... 360.46881652 259.52167074
  276.59536476]
 [900.25914227 420.8054589  681.97502979 ... 368.18083924  78.35759038
  127.26538846]
 [903.12936843 423.35526247 67

0
[[184.28253873 246.85080193 223.81337237 ... 281.07577007 299.89120667
  298.93682798]
 [367.04288153 265.769757   368.55871677 ... 426.63242361 445.94918065
  442.68752873]
 [267.19666329 236.4750507  158.11766291 ... 275.8592253  298.07659094
  296.72836738]
 ...
 [275.84216511 236.677256   201.27732481 ...  95.26412927  70.62910091
   66.32066529]
 [286.54534225 260.65068523 201.3458178  ...  85.77380327   9.98061271
   12.54536616]
 [283.83839409 258.57368184 197.15237973 ...  81.83875879  12.14203394
    8.2779944 ]]
[[618.91544283 573.1778261  633.81690254 ... 783.45273438 700.87592143
  707.30720739]
 [789.9779506  377.18450023 581.87969258 ... 522.24631138 344.66241024
  361.09452591]
 [745.04075643 543.64685785 382.19796979 ... 717.72430541 633.66547356
  636.75031104]
 ...
 [891.11884631 456.95674728 697.63348088 ... 360.46881652 262.94515342
  278.83581196]
 [922.67913098 437.994492   695.40225716 ... 371.60432191  78.35759038
  128.44842393]
 [924.36632166 439.36126009 69

0
[[184.28253873 252.08175754 232.80613371 ... 300.0722761  322.31119538
  320.17378121]
 [372.27383714 265.769757   372.32052249 ... 440.39797403 463.13821375
  458.69352635]
 [276.18942463 240.23685643 158.11766291 ... 285.86296999 311.50381831
  308.97255928]
 ...
 [294.83867115 250.44280642 211.2810695  ...  95.26412927  74.05258358
   68.56111249]
 [308.96533095 277.83971833 214.77304517 ...  89.19728594   9.98061271
   13.72840163]
 [305.07534732 274.57967946 209.39657162 ...  84.07920599  13.32506941
    8.2779944 ]]
[[618.91544283 578.40878171 642.80966388 ... 802.44924041 723.29591014
  728.54416062]
 [795.20890621 377.18450023 585.64149831 ... 536.0118618  361.85144334
  377.10052353]
 [754.03351777 547.40866358 382.19796979 ... 727.72805011 647.09270093
  648.99450293]
 ...
 [910.11535234 470.7222977  707.63722557 ... 360.46881652 266.36863609
  281.07625915]
 [945.09911969 455.18352509 708.82948453 ... 375.02780459  78.35759038
  129.63145941]
 [945.6032749  455.36725771 70

In [17]:
cwd='/Users/emilyolafson/GIT/stroke-graph-matching'
data_dir = str(cwd) + '/results/'
counter = 0
for i in session_swaps:
    swaps=i[:][:][:]
    meanswaps=np.mean(swaps,axis=2)
    mdic = {"meanswaps": meanswaps}
    sio.savemat(data_dir + 'stroke_meanswaps' + str(counter) + '.mat', mdic)
    counter = counter +1
    